In [58]:
LOGDIR = "../logfiles/2020-11-17/"
import os
import re
import pandas as pd

from IPython.display import display

#2020-11-17 22:58:38,135 INFO     Different evaluated sequences: 202
#2020-11-17 22:58:38,222 INFO     Candidate sequences: 100
#2020-11-17 22:58:38,480 INFO     Solution value  = 0.0

ATRIBUTES_TABLE = ["track", "domain", "evaluated_sequences", "candidate_sequences", "solution_value", "sart_runtimes", "num_sequences"]

regex_int = [re.compile(x) for x in [".*Different evaluated sequences: (?P<evaluated_sequences>(\d+))",
                                     ".*Candidate sequences: (?P<candidate_sequences>(\d+))", 
                                    ]]
regex_float = [re.compile(x) for x in [".*Solution value  = (?P<solution_value>(.*))",]]
DATA_LOGS = []
table = []

for logfile in sorted(os.listdir(LOGDIR)):
    _, track, domain = logfile.split("_")
    f = open(f"{LOGDIR}/{logfile}")
    content = f.readlines()
    
    data = { "track" : track, "domain" : domain, "sart_runtimes" : False, "num_sequences" : 0}
    
    for l in content: 
        if "Using sart runtimes on CPLEX optimization" in l: 
            data["sart_runtimes"] = True
            continue
        if "Selected: sequence" in l:
            data["num_sequences"] += 1

            
        for reg in regex_int: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = int(value)
        
        for reg in regex_float: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = float(value)
        
        
    table.append([data[atr] for atr in ATRIBUTES_TABLE])

    DATA_LOGS.append(data)
    
pd.options.display.float_format = "{:,.2g}".format
pd.set_option('display.max_rows', 500)

                 
df = pd.DataFrame(table,columns =  ATRIBUTES_TABLE)
display(df)


            

,track,domain,evaluated_sequences,candidate_sequences,solution_value,sart_runtimes,num_sequences
0,opt14,barman,33,18,63,True,4
1,opt14,blocksworld,18,1,1.7e+16,True,1
2,opt14,childsnack,94,33,42,True,3
3,opt14,data-network,251,180,18,True,2
4,opt14,depots,1008,100,37,True,4
5,opt14,driverlog,3379,100,1,True,3
6,opt14,elevators,198,68,14,True,3
7,opt14,floortile,261,33,54,True,3
8,opt14,grid,319,178,0,True,4
9,opt14,gripper,1602,89,4.4e+05,False,1
